In [1]:
#pip install transformers torch

In [2]:
from transformers import pipeline

# Load the sentiment analysis pipeline
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is 

In [3]:
import pandas as pd
import re

# Example DataFrame (replace with your dataset)
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SMA/instagram_cleaned.csv")

def clean_caption(caption):
    # Remove emojis
    caption = re.sub(r'[^\w\s,.?!]', '', caption)
    # Remove URLs
    caption = re.sub(r'http\S+|www\S+|https\S+', '', caption, flags=re.MULTILINE)
    # Lowercase the caption
    caption = caption.lower().strip()
    return caption

# Apply cleaning function to captions
df['cleaned_caption'] = df['caption'].fillna("").apply(clean_caption)

In [4]:
# Truncate cleaned captions to 512 characters
df['cleaned_caption'] = df['cleaned_caption'].apply(lambda x: x[:512])

In [5]:
# Apply the sentiment analysis model to each cleaned caption
df['sentiment_result'] = df['cleaned_caption'].apply(lambda x: sentiment_model(x))

# Extract the label and score from the result
df['sentiment_label'] = df['sentiment_result'].apply(lambda x: x[0]['label'])
df['sentiment_score'] = df['sentiment_result'].apply(lambda x: x[0]['score'])

In [6]:
df.head(3)

,owner_username,caption,comments,likes,followers,following,created_at,is_video,hour,day_of_week,cleaned_caption,sentiment_result,sentiment_label,sentiment_score
0,christendominique,I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...,268,16382,2144626.0,1021.0,2024-03-01 20:59:18,False,20.0,Friday,im a brunch iced coffee girlie \n\ntop ta3 x ...,"[{'label': 'NEGATIVE', 'score': 0.995167970657...",NEGATIVE,0.995168
1,christendominique,😮‍💨Brow tips I really wish I would have know w...,138,9267,2144626.0,1021.0,2024-02-29 21:10:48,True,21.0,Thursday,brow tips i really wish i would have know when...,"[{'label': 'NEGATIVE', 'score': 0.986371934413...",NEGATIVE,0.986372
2,christendominique,OMG I can’t believe it’s already been 1 yr sin...,1089,10100,2144626.0,1021.0,2024-02-28 21:11:47,True,21.0,Wednesday,omg i cant believe its already been 1 yr since...,"[{'label': 'NEGATIVE', 'score': 0.946092426776...",NEGATIVE,0.946092


In [7]:
df.to_csv("sentiment_data.csv", index=False)